In [1]:
import pathlib
import regex
from requests import get
from zipfile import ZipFile
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns


In [2]:
pd.options.mode.copy_on_write = True

**Download & Initial EDA**

In [3]:
year = 2025
url = f'https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-{year}.zip'

request = get(url)
# Create a folder for the datasets.
try:
    pathlib.Path('datasets/project2').mkdir()
except FileExistsError:
    print('Folder already exist')
# Save the zip file on the disc.
with open('datasets/project2/dataset.zip','wb') as f:
    f.write(request.content)
# Unpack the file on the disc.
with ZipFile('datasets/project2/dataset.zip', 'r') as z:
    try:
        z.extractall('datasets/project2/')
    except FileExistsError:
        print('File/s  already exist')

Folder already exist


**Task 1**

In [4]:
df_public = pd.read_csv('datasets/project2/survey_results_public.csv', low_memory=False)

In [5]:
df_schema = pd.read_csv('datasets/project2/survey_results_schema.csv')

In [6]:
print(f'Data frame shape: {df_public.shape}')

print(f'Data types count:\n{df_public.dtypes.value_counts()}')


Data frame shape: (49191, 172)
Data types count:
object     119
float64     52
int64        1
Name: count, dtype: int64


In [7]:
df_public.head()

,ResponseId,MainBranch,Age,EdLevel,Employment,EmploymentAddl,WorkExp,LearnCodeChoose,LearnCode,LearnCodeAI,...,AIAgentOrchestration,AIAgentOrchWrite,AIAgentObserveSecure,AIAgentObsWrite,AIAgentExternal,AIAgentExtWrite,AIHuman,AIOpen,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,25-34 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Employed,"Caring for dependents (children, elderly, etc.)",8.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,Vertex AI,NaN,NaN,NaN,ChatGPT,NaN,When I don’t trust AI’s answers,"Troubleshooting, profiling, debugging",61256.0,10.0
1,2,I am a developer by profession,25-34 years old,"Associate degree (A.A., A.S., etc.)",Employed,NaN,2.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers;When I want to...,All skills. AI is a flop.,104413.0,9.0
2,3,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Independent contractor, freelancer, or self-em...",None of the above,10.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code;GitHub Copilot;Google Gemini,NaN,When I don’t trust AI’s answers;When I want to...,"Understand how things actually work, problem s...",53061.0,8.0
3,4,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,None of the above,4.0,"Yes, I am not new to coding but am learning ne...","Other online resources (e.g. standard search, ...","Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code,NaN,When I don’t trust AI’s answers;When I want to...,NaN,36197.0,6.0
4,5,I am a developer by profession,35-44 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...","Caring for dependents (children, elderly, etc.)",21.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers,"critical thinking, the skill to define the tas...",60000.0,7.0


In [8]:
df_public.describe()

,ResponseId,WorkExp,YearsCode,TechEndorse_1,TechEndorse_2,TechEndorse_3,TechEndorse_4,TechEndorse_5,TechEndorse_6,TechEndorse_7,...,SO_Actions_3,SO_Actions_4,SO_Actions_5,SO_Actions_6,SO_Actions_9,SO_Actions_7,SO_Actions_10,SO_Actions_15,ConvertedCompYearly,JobSat
count,49191.000000,42893.000000,43042.000000,35975.000000,35975.000000,35975.000000,35975.000000,35975.000000,35975.000000,35975.000000,...,26260.000000,26260.000000,26260.000000,26260.000000,26260.000000,26260.000000,26260.000000,26260.000000,2.394700e+04,26670.000000
mean,24596.000000,13.367403,16.570861,7.867352,4.104211,4.110271,5.678193,4.119388,5.225990,6.477387,...,5.718355,4.561767,4.790861,5.199657,5.676314,4.984653,7.099505,10.079284,1.017615e+05,7.201950
std,14200.362883,10.800117,11.787610,2.397432,2.275821,2.329536,2.398084,2.437945,2.801045,2.331468,...,2.628016,3.070548,2.643177,2.563562,2.310659,2.490095,2.469394,1.940928,4.617569e+05,1.997245
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00,0.000000
25%,12298.500000,5.000000,8.000000,7.000000,2.000000,2.000000,4.000000,2.000000,3.000000,5.000000,...,3.000000,1.000000,3.000000,3.000000,4.000000,3.000000,6.000000,10.000000,3.817100e+04,6.000000
50%,24596.000000,10.000000,14.000000,9.000000,4.000000,4.000000,6.000000,4.000000,5.000000,7.000000,...,6.000000,4.000000,5.000000,5.000000,6.000000,5.000000,8.000000,10.000000,7.532000e+04,8.000000
75%,36893.500000,20.000000,24.000000,9.000000,6.000000,6.000000,8.000000,6.000000,8.000000,8.000000,...,8.000000,7.000000,7.000000,7.000000,7.000000,7.000000,9.000000,10.000000,1.205960e+05,8.000000
max,49191.000000,100.000000,100.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,5.000000e+07,10.000000


In [9]:
print(f'Data frame shape: {df_schema.shape}')

print(f'Data types count:\n{df_schema.dtypes.value_counts()}')

Data frame shape: (139, 6)
Data types count:
object     5
float64    1
Name: count, dtype: int64


In [10]:
df_schema.head()

,qid,qname,question,type,sub,sq_id
0,QID18,TechEndorse_1,What attracts you to a technology or causes yo...,RO,AI integration or AI Agent capabilities,1.0
1,QID18,TechEndorse_2,What attracts you to a technology or causes yo...,RO,Easy-to-use API,2.0
2,QID18,TechEndorse_3,What attracts you to a technology or causes yo...,RO,Robust and complete API,3.0
3,QID18,TechEndorse_4,What attracts you to a technology or causes yo...,RO,Customizable and manageable codebase,4.0
4,QID18,TechEndorse_5,What attracts you to a technology or causes yo...,RO,Reputation for quality,5.0


In [11]:
print(f'Public shape before cleaning: {df_public.shape}\nSchema shape before cleaning: {df_schema.shape}')
df_public_clear = df_public.drop_duplicates()
df_schema_clear = df_schema.drop_duplicates()
print(f'Public shape: {df_public_clear.shape}\nSchema shape: {df_schema_clear.shape}')
df_public_clear.isna().mean()

Public shape before cleaning: (49191, 172)
Schema shape before cleaning: (139, 6)
Public shape: (49191, 172)
Schema shape: (139, 6)


ResponseId             0.000000
MainBranch             0.000000
Age                    0.000000
EdLevel                0.021183
Employment             0.017320
                         ...   
AIAgentExtWrite        0.982537
AIHuman                0.406517
AIOpen                 0.541786
ConvertedCompYearly    0.513183
JobSat                 0.457828
Length: 172, dtype: float64

To count the number of respondents, we must assume that every response was made by a unique respondent. 

In [12]:
print(f'The number of respondents: {df_public_clear['ResponseId'].nunique()}')

The number of respondents: 49191


**Task 2**

In [283]:
questions = df_schema_clear['qname'].tolist()   # Packing question names in a list.
colons = df_public_clear.columns.tolist()
print(questions, '\n')
print(colons)

['TechEndorse_1', 'TechEndorse_2', 'TechEndorse_3', 'TechEndorse_4', 'TechEndorse_5', 'TechEndorse_6', 'TechEndorse_7', 'TechEndorse_8', 'TechEndorse_9', 'TechEndorse_13', 'TechEndorse_13_TEXT', 'TechOppose_1', 'TechOppose_2', 'TechOppose_3', 'TechOppose_5', 'TechOppose_7', 'TechOppose_9', 'TechOppose_11', 'TechOppose_13', 'TechOppose_16', 'TechOppose_15', 'TechOppose_15_TEXT', 'JobSatPoints_1', 'JobSatPoints_2', 'JobSatPoints_16', 'JobSatPoints_3', 'JobSatPoints_4', 'JobSatPoints_5', 'JobSatPoints_6', 'JobSatPoints_7', 'JobSatPoints_8', 'JobSatPoints_9', 'JobSatPoints_10', 'JobSatPoints_11', 'JobSatPoints_13', 'JobSatPoints_14', 'JobSatPoints_15', 'JobSatPoints_15_TEXT', 'SO_Actions_1', 'SO_Actions_16', 'SO_Actions_3', 'SO_Actions_4', 'SO_Actions_5', 'SO_Actions_6', 'SO_Actions_9', 'SO_Actions_7', 'SO_Actions_10', 'SO_Actions_15', 'SO_Actions_15_TEXT', 'MainBranch', 'Age', 'EdLevel', 'Employment', 'EmploymentAddl', 'WorkExp', 'LearnCodeChoose', 'LearnCode', 'LearnCodeAI', 'AILearnHow'

In [284]:
df_task2 = df_public_clear.copy()

**Missing names of the columns in the results of the survey:** 'Language', 'Database', 'Platform', 'Webframe', 'DevEnvs', 'SOTags', 'OpSys', 'OfficeStackAsync', 'CommPlatform', 'AIModels', 'AITool', 'AIAgentImpact', 'AIAgentChallenges'

The dataset with answers could have a few answer columns for one question (e.g. 'LanguageHaveWorkedWith' and 'LanguageWantToWorkWith' for 'Language')

The first approach to deal with it is to do not use missing columns, which will lead to loss of data.

In [285]:
fix_list = ['Language', 'Database', 'Platform', 'Webframe', 'DevEnvs', 'SOTags', 'OpSys', 'OfficeStackAsync', 'CommPlatform', 'AIModels', 'AITool', 'AIAgentImpact', 'AIAgentChallenges']
for i in fix_list:
    questions.remove(i)

print(f'The number of respondents who answered all question: {df_task2[questions].dropna().shape[0]}')

The number of respondents who answered all question: 0


The second approach is to use multiple answer columns instead of a single question name, which increase the chance to meet non-value in the row.

In [286]:
questions = df_schema_clear['qname'].tolist()
fix_list = ['Language', 'Database', 'Platform', 'Webframe', 'DevEnvs', 'SOTags', 'OfficeStackAsync', 'CommPlatform', 'AIModels']
for i in fix_list:
    questions.remove(i)
    questions.append(i + 'HaveWorkedWith')
    questions.append(i + 'WantToWorkWith')

questions.remove('OpSys')
questions.append('OpSysPersonal use')
questions.append('OpSysProfessional use')

**Missing names of the columns in the results of the survey:** 'AITool', 'AIAgentImpact', 'AIAgentChallenges'

In [287]:
fix_list = ['AITool', 'AIAgentImpact', 'AIAgentChallenges']

for i in fix_list:
    for col in df_public_clear.columns:
        if regex.match(f'{i}.*', col):
            questions.append(col)
    questions.remove(i)

df_task2 = df_task2[questions]

In [288]:
print(f'The number of respondents who answered all question: {df_task2[questions].dropna().shape[0]}')

The number of respondents who answered all question: 0


There is also the third approach, filling NaN value and aggregate strings into a new column with a missing name. It will increase data quality. However, no one pays me for doing that.

**Task 3**

In [289]:
df_task3 = df_public_clear['WorkExp'].copy()
df_task3 = df_task3.dropna()
iqr = df_task3.quantile(0.75) - df_task3.quantile(0.25)
lower_bound = df_task3.quantile(0.25) - iqr * 1.5
upper_bound = df_task3.quantile(0.75) + iqr * 1.5

print(f'IQR: {iqr}')
print(f'Mean equal: {df_task3.mean()}')
print(f'Mean without outliers: {df_task3[(df_task3 < upper_bound) & (df_task3 > lower_bound)].mean()}')
print(f'Median equal: {df_task3.median()}')
print(f'Mode equal: {df_task3.mode()[0]}')

IQR: 15.0
Mean equal: 13.367402606485907
Mean without outliers: 12.819841551377557
Median equal: 10.0
Mode equal: 10.0


**Task 4**

In [290]:
df_task4 = df_public_clear['RemoteWork'].copy()
df_task4 = df_task4.dropna()

print(f'Number of the people who work remotely: {df_task4.value_counts()['Remote']}')

Number of the people who work remotely: 10931


**Task 5**

In [291]:
df_task5 = df_public_clear[['ResponseId', 'LanguageHaveWorkedWith']]
df_task5 = df_task5.dropna(subset=['LanguageHaveWorkedWith']).reset_index(drop=True)
df_task5['LanguageHaveWorkedWith'] = df_task5['LanguageHaveWorkedWith'].apply(lambda x: x.split(';'))
df_task5 = df_task5.explode(column='LanguageHaveWorkedWith').reset_index(drop=True)
print(f'Share of people who use Python: {(df_task5[(df_task5['LanguageHaveWorkedWith'] == 'Python')]['ResponseId'].nunique() / df_task5['ResponseId'].unique().size * 100):1.2f}%')

Share of people who use Python: 58.13%


**Task 6**

In [292]:
df_task6 = pd.DataFrame()
for i in colons:
    if regex.match('LearnCode.*', i):
        df_task6[i] = df_public_clear[i].copy()
print(f'List of colons: {df_task6.columns.values}')
df_task6['ResponseId'] = df_public_clear['ResponseId'].copy()
df_task6 = df_task6[['LearnCode', 'ResponseId']].dropna(subset='LearnCode').reset_index(drop=True)
df_task6['LearnCode'] = df_task6['LearnCode'].apply(lambda x: x.split(';'))
df_task6 = df_task6.explode(column='LearnCode').reset_index(drop=True)
print(f'People who finished online courses: {df_task6[df_task6['LearnCode'] == 'Online Courses or Certification (includes all media types)'].count().iloc[0]}')

List of colons: ['LearnCodeChoose' 'LearnCode' 'LearnCodeAI']
People who finished online courses: 10973


**Task 7**

In [293]:
df_task7 = df_public_clear[['ResponseId', 'LanguageHaveWorkedWith', 'ConvertedCompYearly', 'Country']]
df_task7 = df_task7.dropna(subset=['LanguageHaveWorkedWith']).reset_index(drop=True)
df_task7['LanguageHaveWorkedWith'] = df_task7['LanguageHaveWorkedWith'].apply(lambda x: x.split(';'))
df_task7 = df_task7.explode(column='LanguageHaveWorkedWith').reset_index(drop=True)

In [294]:
df_task7 = df_task7[df_task7['LanguageHaveWorkedWith'] == 'Python'].reset_index(drop=True)
df_task7['ConvertedCompYearly'] = df_task7['ConvertedCompYearly'].fillna(df_task7['ConvertedCompYearly'].median(skipna=True))

iqr = df_task7['ConvertedCompYearly'].quantile(0.75) - df_task7['ConvertedCompYearly'].quantile(0.25)
lower_bound = df_task7['ConvertedCompYearly'].quantile(0.25) - iqr * 1.5
upper_bound = df_task7['ConvertedCompYearly'].quantile(0.75) + iqr * 1.5

df_task7 = df_task7[(df_task7['ConvertedCompYearly'] < upper_bound) & (df_task7['ConvertedCompYearly'] > lower_bound)].reset_index(drop=True)

In [295]:
df_task7.groupby('Country')['ConvertedCompYearly'].agg(['mean', 'median']).round(2)

,mean,median
Country,,
Afghanistan,35953.50,33000.0
Albania,68912.56,76828.0
Algeria,59589.52,76828.0
Andorra,52207.00,52207.0
Angola,76828.00,76828.0
...,...,...
"Venezuela, Bolivarian Republic of...",27365.87,12000.0
Viet Nam,52012.09,76828.0
Yemen,47562.33,56934.5


**Task 8**

In [296]:
df_task8 = df_public_clear[['ResponseId', 'ConvertedCompYearly', 'EdLevel']]
df_task8['ConvertedCompYearly'] = df_task8['ConvertedCompYearly'].fillna(df_task8['ConvertedCompYearly'].median(skipna=True))
df_task8['ConvertedCompYearly'] = df_task8['ConvertedCompYearly'].sort_values(ascending=False).reset_index(drop=True)
df_task8 = df_task8.head(5)

In [297]:
df_task8.head()

,ResponseId,ConvertedCompYearly,EdLevel
0,1,50000000.0,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"
1,2,33552715.0,"Associate degree (A.A., A.S., etc.)"
2,3,18387548.0,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
3,4,15430267.0,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)"
4,5,13921760.0,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)"


**Task 9**

In [75]:
df_task9 = df_public_clear.copy()

In [76]:
df_task9 = df_task9[['ResponseId', 'Age', 'LanguageHaveWorkedWith']].dropna()
df_task9['LanguageHaveWorkedWith'] = df_task9['LanguageHaveWorkedWith'].apply(lambda x: x.split(';'))
df_task9['Age'] = df_task9['Age'].astype('category')
df_task9['Age'] = df_task9['Age'].cat.as_ordered()

In [77]:
df_task9_final = df_task9.groupby('Age')['LanguageHaveWorkedWith'].count().reset_index(name='Total')

C:\Users\morov\AppData\Local\Temp\ipykernel_324\2200583687.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_task9_final = df_task9.groupby('Age')['LanguageHaveWorkedWith'].count().reset_index(name='Total')


In [81]:
df_task9 = df_task9.explode(column='LanguageHaveWorkedWith').reset_index(drop=True)
df_task9_final['Python'] = df_task9[df_task9['LanguageHaveWorkedWith'] == 'Python'].groupby('Age')['LanguageHaveWorkedWith'].count().reset_index(name='Python').loc[:, 'Python']
df_task9_final.head()

C:\Users\morov\AppData\Local\Temp\ipykernel_324\1234882703.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_task9_final['Python'] = df_task9[df_task9['LanguageHaveWorkedWith'] == 'Python'].groupby('Age')['LanguageHaveWorkedWith'].count().reset_index(name='Python').loc[:, 'Python']


,Age,Total,Python
0,18-24 years old,5126,3678
1,25-34 years old,10257,6082
2,35-44 years old,8965,4849
3,45-54 years old,4483,2413
4,55-64 years old,1986,976


In [83]:
df_task9_final['PythonShare'] = (df_task9_final['Python'] / df_task9_final['Total'] * 100).round(2)
df_task9_final.head(10)

,Age,Total,Python,PythonShare
0,18-24 years old,5126,3678,71.75
1,25-34 years old,10257,6082,59.30
2,35-44 years old,8965,4849,54.09
3,45-54 years old,4483,2413,53.83
4,55-64 years old,1986,976,49.14
5,65 years or older,678,295,43.51
6,Prefer not to say,176,117,66.48


**Task 10**

In [97]:
df_task10 = df_public_clear.copy()

In [98]:
df_task10 = df_task10[['ResponseId', 'RemoteWork', 'ConvertedCompYearly', 'Industry']]
df_task10 = df_task10[(df_task10['RemoteWork'] == 'Remote') & (df_task10['ConvertedCompYearly'] >= df_task10['ConvertedCompYearly'].quantile(0.75))].reset_index(drop=True)
df_task10 = df_task10.drop(columns=['RemoteWork'])
df_task10 = df_task10.dropna(subset=['Industry']).groupby('Industry')['ResponseId'].count().reset_index(name='Count').sort_values(by='Count', ascending=False)

In [100]:
df_task10.head(5)

,Industry,Count
13,Software Development,1186
3,Fintech,190
5,Healthcare,188
11,Other:,176
8,"Internet, Telecomm or Information Services",138
